In [4]:
from main import *
from data_generation import *

# Synthetic Dataset Generation

In [5]:
tasks = {'log_in': ['one',['credentials check','sing up','recover pw and log in'],(0,10)],
            'search_book': ['rand',['history','fantasy','crime','poetry','biography'],(5,15)],
            'shipment' : ['all',['adress','door number','zip code'],(15,50)],
            'payment' : ['one',['visa','master card','revolut','paypal','apple pay'],(10,100)],
            'new_site' : ['con',['site1','site2'],(10,100)],
            'site1' : ['opt',['site1_1','site1_2','site1_3'],(10,100)],
            'site2' : ['opt',['site2_1','site2_2'],(10,100)]
}   
start_time = datetime(2024, 6, 3, 9, 0, 0)  
end_time = datetime(2024, 6, 3, 10, 45, 0) 
 
spark = SparkSession.builder.getOrCreate()
# generate_dataset(tasks, 1000000,start_time,end_time,file_name="dataset")
# data = spark.read.csv("data/SDG_dataset.csv", header=True, inferSchema=True)  
# generate_dataset(tasks, 100000,start_time,end_time,file_name="dataset_test")  
data = spark.read.csv("data/SDG_dataset_test.csv", header=True, inferSchema=True)

# Part 1

k = 3

In [6]:
exp3 = experiment(data,3)

average trace length: 38.158701120512234
average trace # shingles: 28.111136519551795
Initial number of cases: 4373
get the threshold
threshold: 0.9642857142857143
Number of unique processes after merging them with 0.9642857142857143 threshold using 7-shingles: 3614
######################### 3737 vs 594 ################################
jaccard similarity: 0.9259259259259259
3737 : S0S3S3_1S3_2S3_3S5S5_1S5_1_2S4S4_2
594 : S0S3S3_1S3_2S3_3S5S5_1S5_1_2S4S4_3
#######################################################################


k = 4

In [ ]:
exp4 = experiment(data,4)

k = 5

In [ ]:
exp5 = experiment(data,5)

k = 6

In [ ]:
exp6 = experiment(data,6)

k = 7

In [ ]:
exp7 = experiment(data,7)

k = 8

In [ ]:
exp8 = experiment(data,8)

k = 9

In [ ]:
exp9 = experiment(data,9)

# Part 2

### Approach 1

threshold 0.7

In [ ]:
experiment(data,7,threshold=0.9)

### Approach 2

In [ ]:
#experiment(data,3,approach=2)

## Perfomance evaluation